In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import numpy as np
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/AMZN (1).csv")
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-03-24,163.749496,164.118500,160.050003,163.649506,163.649506,56798000
1,2022-03-25,164.000000,165.368500,162.250000,164.773499,164.773499,49032000
2,2022-03-28,164.975006,169.037506,164.899994,168.990494,168.990494,59854000
3,2022-03-29,170.384003,170.831497,167.868500,169.315002,169.315002,66154000
4,2022-03-30,168.509506,168.950500,165.500000,166.300995,166.300995,56168000
...,...,...,...,...,...,...,...
246,2023-03-17,99.790001,100.660004,97.459999,98.949997,98.949997,87173200
247,2023-03-20,98.410004,98.480003,95.699997,97.709999,97.709999,62388900
248,2023-03-21,98.139999,100.849998,98.000000,100.610001,100.610001,58597300
249,2023-03-22,100.449997,102.099998,98.610001,98.699997,98.699997,57475400


In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']])

# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        seq = data[i:i+seq_length]
        label = data[i+seq_length-1]
        sequences.append((seq, label))
    return sequences

seq_length = 10
sequences = create_sequences(data_scaled, seq_length)

# Build the LSTM model
model = keras.Sequential([
    layers.LSTM(50, activation='relu', input_shape=(seq_length, data_scaled.shape[1])),
    layers.Dense(data_scaled.shape[1])
])
model.compile(optimizer='adam', loss='mse')

# Convert sequences to arrays
X = np.array([seq for seq, label in sequences])
y = np.array([label for seq, label in sequences])

# Train the model without printing progress
model.fit(X, y, epochs=50, batch_size=64, verbose=0)

# Make predictions
predictions = model.predict(X)

# Inverse transform to get actual values
predictions_actual = scaler.inverse_transform(predictions)
y_actual = scaler.inverse_transform(y)

# Create a DataFrame to hold results
results = pd.DataFrame({
    'Date': data['Date'].values[seq_length - 1:],
    'Actual_Close': y_actual[:, 3],
    'Predicted_Close': predictions_actual[:, 3]
})

# Print the results
print(results)

8/8 [==============================] - 0s 3ms/step
           Date  Actual_Close  Predicted_Close
0    2022-04-06    158.755997       160.775711
1    2022-04-07    157.784500       156.882935
2    2022-04-08    154.460495       153.266830
3    2022-04-11    151.121994       150.639084
4    2022-04-12    150.787506       148.904022
..          ...           ...              ...
237  2023-03-17     98.949997        97.367172
238  2023-03-20     97.709999        97.969299
239  2023-03-21    100.610001        99.020653
240  2023-03-22     98.699997        99.490051
241  2023-03-23     98.709999        99.639153

[242 rows x 3 columns]
